In [ ]:
경력0년을 삭제하지 말고 신입으로 변경하면 사전 크기 유지되잖아?
그러면 rows데이터에서 받아온 값이 0이면 신입으로 변경된 후 gpt4 모델에게 프롬프트로 전달 되는 거야?

In [ ]:
네, 그렇게 하면 rows 데이터에서 받아온 값이 0이면 "신입"으로 변경된 후, GPT-4 모델에게 전달됩니다.

In [ ]:
# 2월 7일 오전에 할 것 
이미지파일 경로 유지한 채, 2번째 원소 값 변경하기 


In [ ]:
import openai
import sqlite3
import ast  # 문자열로 저장된 리스트 변환용

# ✅ OpenAI API 키 설정

# ✅ SQLite 데이터베이스 연결
db_path = "c:/Users/asia/Desktop/asia_camp_2024/final_project_asia/3.ETL_DB_Load(Career,Language)/asia (2).db"
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# ✅ `duty_analysis` 테이블에서 career 데이터 조회
cursor.execute("SELECT duty, career FROM duty_analysis;")
rows = cursor.fetchall()

# ✅ 직무별 연차별 채용 데이터 조회
cursor.execute("""
    SELECT duty, career, COUNT(*) as job_count
    FROM processing
    GROUP BY duty, career
    ORDER BY duty, career;
""")
career_data = cursor.fetchall()

# ✅ 데이터 변환 (직무별 연차별 데이터 정리)
job_career_dict = {}
for duty, career, job_count in career_data:
    if duty not in job_career_dict:
        job_career_dict[duty] = []
    # ✅ 경력 0년 → "신입"으로 변환 (하지만 기존 0 삭제하지 않음 → 사전 크기 유지)
    career_label = "신입" if career == 0 else f"{career}년차"
    job_career_dict[duty].append((career_label, job_count))

# ✅ 직무별 GPT-3.5 요약 생성 및 업데이트
for duty, career in rows:
    if duty not in job_career_dict:
        continue  # 해당 직무에 대한 데이터가 없으면 건너뜀

    # ✅ 프롬프트 생성
    job_details = "\n".join([f"- {c[0]}: {c[1]}건" for c in job_career_dict[duty]])
    prompt = f"""
    "{duty}" 직무의 채용 공고 데이터를 분석해주세요.

    [직무별 연차별 채용 공고 개수]
    {job_details}

    위 데이터를 기반으로 다음을 분석해주세요:
    1. 가장 많이 요청된 경력 연차와 해당 채용 공고 수
    2. 두 번째로 많이 요청된 경력 연차와 해당 채용 공고 수
    3. 최소 요구되는 연차
    4. {duty} 직무에서의 경력별 채용 특징 및 트렌드 분석

    ❗ 단, "신입"이라는 표현을 사용하며, "경력 0년차"라는 표현은 하지 마세요.
    ❗ 모든 문장은 "입니다"로 끝나도록 통일해주세요.
    ❗ 설명은 간결하게 요약해서 제공해주세요.
    """

    try:
        # ✅ GPT-3.5 API 호출
        response = openai.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "user", "content": prompt}]
        )
        gpt3_summary = response.choices[0].message.content.strip()

        # ✅ 기존 career 컬럼 값 가져오기 (이미지 경로 유지)
        cursor.execute("SELECT career FROM duty_analysis WHERE duty = ?;", (duty,))
        existing_career_data = cursor.fetchone()

        if existing_career_data and existing_career_data[0]:
            try:
                # career 필드가 문자열 형태의 리스트라면 변환
                career_list = ast.literal_eval(existing_career_data[0])
                if isinstance(career_list, list) and career_list:
                    image_path = career_list[0]  # 첫 번째 원소(이미지 경로) 유지
                else:
                    image_path = ""
            except:
                image_path = existing_career_data[0]  # 예외 발생 시 기존 데이터 유지
        else:
            image_path = ""

        # ✅ `career` 값 업데이트 (['이미지 경로', 'GPT-3.5 설명'])
        career_list = [image_path, gpt3_summary] if image_path else [gpt3_summary]
        updated_text = str(career_list)  # 리스트를 문자열로 변환하여 저장

        cursor.execute(
            "UPDATE duty_analysis SET career = ? WHERE duty = ?;",
            (updated_text, duty)
        )
        conn.commit()

        print(f"✅ {duty} 요약 완료 (GPT-3.5 적용): {gpt3_summary}")

    except Exception as e:
        print(f"❌ {duty} 요약 실패: {e}")

# ✅ DB 연결 종료
conn.close()
print("🎯 모든 직무의 경력 요구 사항 요약이 GPT-3.5로 업데이트되었습니다.")


✅ AI 요약 완료 (GPT-3.5 적용): 1. 가장 많이 요청된 경력 연차는 3년차로, 해당 채용 공고는 38건 입니다.

2. 두 번째로 많이 요청된 경력 연차는 2년차로, 해당 채용 공고는 25건 입니다.

3. AI 직무에서 최소 요구되는 연차는 3년차 이상으로, 신입과 1년차는 상대적으로 적게 요구됩니다.

4. AI 직무에서는 주로 2~5년차의 경력자를 많이 채용하며, 신입 및 경력 6년차 이상은 적은 수의 채용 공고가 있습니다. 특히 3년차의 수요가 높은 편이며, 경력 레벨이 올라갈수록 채용 공고 수가 감소하는 경향을 보입니다.
✅ ERP 요약 완료 (GPT-3.5 적용): 1. ERP 직무에서 가장 많이 요구되는 경력 연차는 7년차이며, 해당 연차에는 3건의 채용 공고가 있습니다.

2. 두 번째로 많이 요구되는 경력 연차는 3년차이며, 해당 연차에는 3건의 채용 공고가 있습니다.

3. ERP 직무에서의 최소 요구되는 경력은 1년차(신입)입니다.

4. ERP 직무에서는 3년차와 7년차 경력자가 가장 많이 요구되며, 경력이 있는 신입자들에게도 기회가 주어지고 있습니다. 이는 기업이 경력과 능력을 고려하여 적절한 인재를 채용하려는 경향을 보여줍니다.
✅ PM 요약 완료 (GPT-3.5 적용): 1. 가장 많이 요청된 경력 연차는 5년차이며, 해당 채용 공고 수는 22건입니다.

2. 두 번째로 많이 요청된 경력 연차는 3년차이며, 해당 채용 공고 수는 14건입니다.

3. PM 직무에서 최소 요구되는 연차는 1년차이며, 해당 채용 공고 수는 5건입니다.

4. PM 직무에서는 주로 5년차의 경력을 요구하는 채용 공고가 가장 많으며, 1년차와 3년차의 채용 공고도 많이 나옵니다. 신입으로는 별로 요구되지 않지만, 다양한 연차의 경력을 고려하는 경향이 있습니다.
✅ QA 요약 완료 (GPT-3.5 적용): 1. 가장 많이 요청된 경력 연차는 3년차로, 해당 채용 공고 수는 39건입니다.
2. 두 번째로 많이 요청된 경력 연차는 5년차로, 해당 채용

In [ ]:
# vr 직무만 0년차 처리가 되지 않아 실제 processing table의 값 확인 
# 경력 값 모두 int형 확인 

In [3]:
import sqlite3

# ✅ SQLite 데이터베이스 연결
db_path = "c:/Users/asia/Desktop/asia_camp_2024/final_project_asia/3.ETL_DB_Load(Career,Language)/asia (2).db"
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# ✅ career 컬럼의 DISTINCT 값 조회
cursor.execute("SELECT career FROM processing;")
career_values = cursor.fetchall()

# ✅ 각 값의 데이터 타입 확인
for value in career_values:
    print(f"값: {value[0]}, 타입: {type(value[0])}")

# ✅ DB 연결 종료
conn.close()


값: 2, 타입: <class 'int'>
값: 3, 타입: <class 'int'>
값: 3, 타입: <class 'int'>
값: 0, 타입: <class 'int'>
값: 3, 타입: <class 'int'>
값: 2, 타입: <class 'int'>
값: 9, 타입: <class 'int'>
값: 10, 타입: <class 'int'>
값: 3, 타입: <class 'int'>
값: 0, 타입: <class 'int'>
값: 9, 타입: <class 'int'>
값: 5, 타입: <class 'int'>
값: 3, 타입: <class 'int'>
값: 5, 타입: <class 'int'>
값: 2, 타입: <class 'int'>
값: 4, 타입: <class 'int'>
값: 7, 타입: <class 'int'>
값: 4, 타입: <class 'int'>
값: 5, 타입: <class 'int'>
값: 4, 타입: <class 'int'>
값: 7, 타입: <class 'int'>
값: 0, 타입: <class 'int'>
값: 15, 타입: <class 'int'>
값: 10, 타입: <class 'int'>
값: 3, 타입: <class 'int'>
값: 3, 타입: <class 'int'>
값: 3, 타입: <class 'int'>
값: 3, 타입: <class 'int'>
값: 5, 타입: <class 'int'>
값: 5, 타입: <class 'int'>
값: 3, 타입: <class 'int'>
값: 7, 타입: <class 'int'>
값: 5, 타입: <class 'int'>
값: 5, 타입: <class 'int'>
값: 5, 타입: <class 'int'>
값: 5, 타입: <class 'int'>
값: 5, 타입: <class 'int'>
값: 5, 타입: <class 'int'>
값: 6, 타입: <class 'int'>
값: 5, 타입: <class 'int'>
값: 5, 타입: <class 'int'>
값: 5, 타입: <cl

In [5]:
import sqlite3

# ✅ SQLite 데이터베이스 연결
db_path = "c:/Users/asia/Desktop/asia_camp_2024/final_project_asia/3.ETL_DB_Load(Career,Language)/asia (2).db"
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

cursor.execute("SELECT career, typeof(career) FROM processing GROUP BY career;")
career_types = cursor.fetchall()

for value, dtype in career_types:
    print(f"값: {value}, 데이터 타입: {dtype}")


# ✅ DB 연결 종료
conn.close()


값: 0, 데이터 타입: integer
값: 1, 데이터 타입: integer
값: 2, 데이터 타입: integer
값: 3, 데이터 타입: integer
값: 4, 데이터 타입: integer
값: 5, 데이터 타입: integer
값: 6, 데이터 타입: integer
값: 7, 데이터 타입: integer
값: 8, 데이터 타입: integer
값: 9, 데이터 타입: integer
값: 10, 데이터 타입: integer
값: 11, 데이터 타입: integer
값: 12, 데이터 타입: integer
값: 13, 데이터 타입: integer
값: 15, 데이터 타입: integer


In [ ]:
import openai
import sqlite3
import ast  # 문자열로 저장된 리스트 변환용

# ✅ OpenAI API 키 설정

# ✅ SQLite 데이터베이스 연결
db_path = "c:/Users/asia/Desktop/asia_camp_2024/final_project_asia/3.ETL_DB_Load(Career,Language)/asia (2).db"
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# ✅ `duty_analysis` 테이블에서 career 데이터 조회
cursor.execute("SELECT duty, career FROM duty_analysis;")
rows = cursor.fetchall()

# ✅ 직무별 연차별 채용 데이터 조회
cursor.execute("""
    SELECT duty, career, COUNT(*) as job_count
    FROM processing
    GROUP BY duty, career
    ORDER BY duty, career;
""")
career_data = cursor.fetchall()

# ✅ 데이터 변환 (직무별 연차별 데이터 정리)
job_career_dict = {}
for duty, career, job_count in career_data:
    if duty not in job_career_dict:
        job_career_dict[duty] = []
    job_career_dict[duty].append((career, job_count))  # ✅ 원본 int 값 유지

# ✅ 직무별 GPT-3.5 요약 생성 및 업데이트
for duty, career in rows:
    if duty not in job_career_dict:
        continue  # 해당 직무에 대한 데이터가 없으면 건너뜀

    # ✅ 프롬프트 생성 시 "0년차" → "신입" 변환
    job_details = "\n".join([
        f"- {'신입' if c[0] == 0 else f'{c[0]}년차'}: {c[1]}건"
        for c in job_career_dict[duty]
    ])

    prompt = f"""
    "{duty}" 직무의 채용 공고 데이터를 분석해주세요.

    [직무별 연차별 채용 공고 개수]
    {job_details}

    위 데이터를 기반으로 다음을 분석해주세요:
    1. 가장 많이 요청된 경력 연차와 해당 채용 공고 수
    2. 두 번째로 많이 요청된 경력 연차와 해당 채용 공고 수
    3. 최소 요구되는 연차
    4. {duty} 직무에서의 경력별 채용 특징 및 트렌드 분석

    ❗ 단, "신입"이라는 표현을 사용하며, "경력 0년차"라는 표현은 하지 마세요.
    ❗ 모든 문장은 "입니다"로 끝나도록 통일해주세요.
    ❗ 설명은 간결하게 요약해서 제공해주세요.
    """


    try:
        # ✅ GPT-3.5 API 호출
        response = openai.chat.completions.create(
            model="gpt-4",
            messages=[{"role": "user", "content": prompt}]
        )
        gpt3_summary = response.choices[0].message.content.strip()

        # ✅ GPT 응답 후처리: "0년차" → "신입"
        gpt3_summary = gpt3_summary.replace("0년차", "신입")

        # ✅ 기존 career 컬럼 값 가져오기 (이미지 경로 유지)
        cursor.execute("SELECT career FROM duty_analysis WHERE duty = ?;", (duty,))
        existing_career_data = cursor.fetchone()

        if existing_career_data and existing_career_data[0]:
            try:
                # career 필드가 문자열 형태의 리스트라면 변환
                career_list = ast.literal_eval(existing_career_data[0])
                if isinstance(career_list, list) and career_list:
                    image_path = career_list[0]  # 첫 번째 원소(이미지 경로) 유지
                else:
                    image_path = ""
            except:
                image_path = existing_career_data[0]  # 예외 발생 시 기존 데이터 유지
        else:
            image_path = ""

        # ✅ `career` 값 업데이트 (['이미지 경로', 'GPT-3.5 설명'])
        career_list = [image_path, gpt3_summary] if image_path else [gpt3_summary]
        updated_text = str(career_list)  # 리스트를 문자열로 변환하여 저장

        cursor.execute(
            "UPDATE duty_analysis SET career = ? WHERE duty = ?;",
            (updated_text, duty)
        )
        conn.commit()

        print(f"✅ {duty} 요약 완료 (GPT-3.5 적용): {gpt3_summary}")

    except Exception as e:
        print(f"❌ {duty} 요약 실패: {e}")

# ✅ DB 연결 종료
conn.close()
print("🎯 모든 직무의 경력 요구 사항 요약이 GPT-4로 업데이트되었습니다.")


✅ AI 요약 완료 (GPT-3.5 적용): 1. 가장 많이 요청된 경력 연차는 3년차로, 해당 채용 공고 수는 38건입니다.
2. 두 번째로 많이 요청된 경력 연차는 2년차로, 해당 채용 공고 수는 25건입니다.
3. 최소 요구되는 연차는 신입입니다.
4. AI 직무에서의 채용 트렌드를 분석하면, 중간 경력(2년차, 3년차)이 가장 많이 요청되는 경향이 있는 반면, 초충 경력자(신입)나 체직 경력자(6년차 이상)에 대한 수요는 상대적으로 적습니다. 이는 AI 분야가 빠르게 발전하고 있으며, 기본기를 바탕으로 최신 기술 트렌드에 민첩하게 대응할 수 있는 중간 경력자를 선호하는 경향을 보여줍니다. 또한, 심층적인 기술 연구나 리더십이 필요한 경우 5년차 정도의 경력자를 채용하는 경우도 종종 있습니다.
✅ ERP 요약 완료 (GPT-3.5 적용): 1. 가장 많이 요청된 경력 연차는 3년차와 7년차로, 각각 해당 채용 공고가 3건입니다.
2. 두 번째로 많이 요청된 경력 연차는 5년차로, 해당 채용 공고가 2건입니다.
3. 최소 요구되는 연차는 1년차입니다.
4. ERP 직무에서의 경력별 채용 특징 및 트렌드 분석 결과, 중간 경력인 3년차와 상대적으로 높은 경력인 7년차를 요구하는 채용 공고가 가장 많았습니다. 이는 ERP 직무가 특정 전문성과 경험이 많이 요구되는 카테고리임을 의미할 수 있습니다. 반면, 초창기 경력인 1년차나 2년차를 요구하는 채용 공고는 상대적으로 적어, ERP 직무는 상대적으로 업무 수행에 경력이 중요하다고 판단됩니다. 이렇듯 ERP 직무에서의 채용 특징은 경력이 높을수록 유리함을 확인할 수 있습니다.
✅ PM 요약 완료 (GPT-3.5 적용): 1. 가장 많이 요청된 경력 연차는 5년차입니다. 해당 채용 공고 수는 총 22건입니다.
2. 두 번째로 많이 요청된 경력 연차는 3년차입니다. 해당 채용 공고 수는 총 14건입니다.
3. 최소 요구되는 연차는 신입입니다. 
4. PM 직무에서의 경력별 채용 특징 및 트렌드를 분석해보면, 대체적으

In [ ]:
# 신입 , gpt 자동 토큰화 및 자동 교정 오류 (오타) sin입 = 오타 교정 완료 
# 하지만 vr 직무만 변경 x (왜 vr 직무만 넘어가는지?)


In [ ]:
import openai
import sqlite3
import ast  # 문자열로 저장된 리스트 변환용

# ✅ OpenAI API 키 설정

# ✅ SQLite 데이터베이스 연결
db_path = "c:/Users/asia/Desktop/asia_camp_2024/final_project_asia/3.ETL_DB_Load(Career,Language)/asia (2).db"
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# ✅ `duty_analysis` 테이블에서 career 데이터 조회
cursor.execute("SELECT duty, career FROM duty_analysis;")
rows = cursor.fetchall()

# ✅ 직무별 연차별 채용 데이터 조회
cursor.execute("""
    SELECT duty, career, COUNT(*) as job_count
    FROM processing
    GROUP BY duty, career
    ORDER BY duty, career;
""")
career_data = cursor.fetchall()

# ✅ 데이터 변환 (직무별 연차별 데이터 정리)
job_career_dict = {}
for duty, career, job_count in career_data:
    if duty not in job_career_dict:
        job_career_dict[duty] = []
    job_career_dict[duty].append((career, job_count))  # ✅ 원본 int 값 유지

# ✅ VR 직무 데이터 확인 (디버깅)
if "VR" in job_career_dict:
    print(f"🎯 VR 직무 원본 데이터: {job_career_dict['VR']}")

# ✅ 직무별 GPT-3.5 요약 생성 및 업데이트
for duty, career in rows:
    if duty not in job_career_dict:
        continue  # 해당 직무에 대한 데이터가 없으면 건너뜀

    # ✅ 프롬프트 생성 시 "0년차" → "신입" 변환
    job_details = "\n".join([
        f"- {'신입사원' if c[0] == 0 else f'{c[0]}년차'}: {c[1]}건"
        for c in job_career_dict[duty]
    ])

    # ✅ VR 직무의 경우 데이터 변환 확인
    if duty == "VR":
        print(f"✅ VR 직무 변환된 프롬프트 데이터:\n{job_details}")

    prompt = f"""
    "{duty}" 직무의 채용 공고 데이터를 분석해주세요.

    [직무별 연차별 채용 공고 개수]
    {job_details}

    위 데이터를 기반으로 다음을 분석해주세요:
    1. 가장 많이 요청된 경력 연차와 해당 채용 공고 수
    2. 두 번째로 많이 요청된 경력 연차와 해당 채용 공고 수
    3. 최소 요구되는 연차
    4. {duty} 직무에서의 경력별 채용 특징 및 트렌드 분석

    ❗ 단, "신입사원"이라는 표현을 사용하며, "경력 0년차"라는 표현은 하지 마세요.
    ❗ 모든 문장은 "입니다"로 끝나도록 통일해주세요.
    ❗ 설명은 간결하게 요약해서 제공해주세요.
    ❗ "신입사원"이라는 단어가 변형되지 않도록 정확하게 출력해주세요.
    """

    try:
        # ✅ GPT-3.5 API 호출
        response = openai.chat.completions.create(
            model="gpt-4",
            messages=[{"role": "user", "content": prompt}]
        )
        gpt3_summary = response.choices[0].message.content.strip()

        # ✅ GPT 응답 후처리: "0년차" → "신입사원" & "sin입" 오류 수정
        gpt3_summary = gpt3_summary.replace("0년차", "신입사원").replace("sin입", "신입사원")

        # ✅ VR 직무 결과 확인
        if duty == "VR":
            print(f"✅ VR 직무 최종 응답:\n{gpt3_summary}")

        # ✅ 기존 career 컬럼 값 가져오기 (이미지 경로 유지)
        cursor.execute("SELECT career FROM duty_analysis WHERE duty = ?;", (duty,))
        existing_career_data = cursor.fetchone()

        if existing_career_data and existing_career_data[0]:
            try:
                # career 필드가 문자열 형태의 리스트라면 변환
                career_list = ast.literal_eval(existing_career_data[0])
                if isinstance(career_list, list) and career_list:
                    image_path = career_list[0]  # 첫 번째 원소(이미지 경로) 유지
                else:
                    image_path = ""
            except:
                image_path = existing_career_data[0]  # 예외 발생 시 기존 데이터 유지
        else:
            image_path = ""

        # ✅ `career` 값 업데이트 (['이미지 경로', 'GPT-3.5 설명'])
        career_list = [image_path, gpt3_summary] if image_path else [gpt3_summary]
        updated_text = str(career_list)  # 리스트를 문자열로 변환하여 저장

        cursor.execute(
            "UPDATE duty_analysis SET career = ? WHERE duty = ?;",
            (updated_text, duty)
        )
        conn.commit()

        print(f"✅ {duty} 요약 완료 (GPT-3.5 적용): {gpt3_summary}")

    except Exception as e:
        print(f"❌ {duty} 요약 실패: {e}")

# ✅ DB 연결 종료
conn.close()
print("🎯 모든 직무의 경력 요구 사항 요약이 GPT-4로 업데이트되었습니다.")


✅ AI 요약 완료 (GPT-3.5 적용): 1. 가장 많이 요청된 경력 연차는 3년차이며, 해당 채용 공고 수는 38건입니다.
2. 두 번째로 많이 요청된 경력 연차는 2년차이며, 해당 채용 공고 수는 25건입니다.
3. AI 직무에서 최소 요구되는 경력은 신입사원이며, 해당 채용 공고 수는 23건입니다.
4. AI 직무에서는 3년차 이상의 경력을 갖는 지원자를 많이 선호하며, 특히 3년차의 수요가 높은 것으로 나타났습니다. 반면에 4년차 이상의 요구는 상대적으로 적은 편입니다.신입사원 채용도 일정 수준 유지되고 있지만, 그외의 연차에서는 공고 수가 감소하는 경향을 보이고 있습니다.
✅ ERP 요약 완료 (GPT-3.5 적용): 1. ERP 직무에서 가장 많이 요청된 경력 연차는 7년차이며, 해당 채용 공고는 3건입니다.

2. 두 번째로 많이 요청된 경력 연차는 3년차이며, 해당 채용 공고는 3건입니다.

3. ERP 직무에서 최소 요구되는 연차는 1년차이며, 해당 채용 공고는 1건입니다.

4. ERP 직무에서는 주로 3년차부터 7년차의 경력을 요구하는 경향이 있으며, 3년차와 7년차의 채용 공고가 많이 나오는 편입니다. 그러나 신입사원에 대한 채용 공고도 존재하므로, 경력이 다양한 인재를 모집하는 것으로 보입니다.
✅ PM 요약 완료 (GPT-3.5 적용): 1. 가장 많이 요청된 경력 연차는 5년차이며, 해당 채용 공고 수는 22건입니다.

2. 두 번째로 많이 요청된 경력 연차는 3년차이며, 해당 채용 공고 수는 14건입니다.

3. PM 직무에서의 최소 요구되는 연차는 신입사원입니다.

4. PM 직무에서는 주로 5년차와 3년차의 경력을 요구하는 채용 공고가 많으며, 신입사원 역시 일정 비율로 고용되고 있는 편입니다. 4년차 이하의 경력 채용은 적은 편이고, 1신입사원 이상의 경력을 요구하는 채용 공고는 드물게 있습니다.
✅ QA 요약 완료 (GPT-3.5 적용): 1. 가장 많이 요청된 경력 연차는 3년차이며, 해당 채용 공고 수는 39건입니다.

In [ ]:
# vr 직무 변경하기 

In [ ]:
import openai
import sqlite3
import ast  # 문자열로 저장된 리스트 변환용

# ✅ OpenAI API 키 설정

# ✅ SQLite 데이터베이스 연결
db_path = "c:/Users/asia/Desktop/asia_camp_2024/final_project_asia/3.ETL_DB_Load(Career,Language)/asia (2).db"
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# ✅ VR 직무 데이터 조회 (WITH 절 활용)
sql_query = """
WITH career_data AS (
    SELECT 'VR' AS category, career FROM processing WHERE duty LIKE '%VR%'
)
SELECT
    category AS 직무,
    career AS 연차,
    COUNT(*) AS 채용_공고_개수
FROM career_data
GROUP BY category, career
ORDER BY career;
"""
cursor.execute(sql_query)
vr_career_data = cursor.fetchall()

# ✅ VR 직무 career 필드 가져오기
cursor.execute("SELECT career FROM duty_analysis WHERE duty = 'VR';")
existing_career_data = cursor.fetchone()

# ✅ 연차별 데이터 변환
if vr_career_data:
    job_details = "\n".join([
        f"- {'신입사원' if c[1] == '0' else f'{c[1]}년차'}: {c[2]}건"
        for c in vr_career_data
    ])
else:
    job_details = "VR 직무의 채용 데이터가 없습니다."

print(f"🎯 VR 직무 데이터 확인:\n{job_details}")

# ✅ GPT-4 프롬프트 생성
prompt = f"""
"VR" 직무의 채용 공고 데이터를 분석해주세요.

[직무별 연차별 채용 공고 개수]
{job_details}

위 데이터를 기반으로 다음을 분석해주세요:
1. 가장 많이 요청된 경력 연차와 해당 채용 공고 수
2. 두 번째로 많이 요청된 경력 연차와 해당 채용 공고 수
3. 최소 요구되는 연차
4. VR 직무에서의 경력별 채용 특징 및 트렌드 분석

❗ 단, "신입사원"이라는 표현을 사용하며, "경력 0년차"라는 표현은 하지 마세요.
❗ 모든 문장은 "입니다"로 끝나도록 통일해주세요.
❗ 설명은 간결하게 요약해서 제공해주세요.
❗ "신입사원"이라는 단어가 변형되지 않도록 정확하게 출력해주세요.
"""

try:
    # ✅ GPT-4 API 호출
    response = openai.chat.completions.create(
        model="gpt-4",
        messages=[{"role": "user", "content": prompt}]
    )
    gpt4_summary = response.choices[0].message.content.strip()

    print(f"✅ VR 직무 최종 응답:\n{gpt4_summary}")

    # ✅ 기존 career 컬럼 값 가져오기 (이미지 경로 유지)
    if existing_career_data and existing_career_data[0]:
        try:
            career_list = ast.literal_eval(existing_career_data[0])  # 문자열 → 리스트 변환
            if isinstance(career_list, list) and career_list:
                image_path = career_list[0]  # 기존 이미지 경로 유지
                career_list = [image_path, gpt4_summary]  # GPT 설명 업데이트
            else:
                career_list = [gpt4_summary]  # 기존 데이터가 리스트가 아니면 새로 생성
        except:
            career_list = [gpt4_summary]  # 변환 실패 시 GPT 설명만 추가
    else:
        career_list = [gpt4_summary]  # 기존 값이 없으면 새 리스트 생성

    # ✅ 업데이트할 값 변환 (리스트 → 문자열)
    updated_text = str(career_list)

    # ✅ `duty_analysis` 테이블에서 `VR` 직무의 `career` 값 업데이트
    cursor.execute("UPDATE duty_analysis SET career = ? WHERE duty = 'VR';", (updated_text,))
    conn.commit()

    print(f"✅ VR 직무 career 업데이트 완료: {updated_text}")

except Exception as e:
    print(f"❌ VR 직무 요약 실패: {e}")

# ✅ DB 연결 종료
conn.close()


🎯 VR 직무 데이터 확인:
- 0년차: 3건
- 2년차: 3건
- 3년차: 5건
- 4년차: 1건
- 5년차: 3건
✅ VR 직무 최종 응답:
1. 가장 많이 요청된 경력 연차는 3년으로, 해당 채용 공고는 총 5건입니다.
2. 두 번째로 많이 요청된 경력 연차는 0년(신입사원), 2년, 5년으로, 각각 해당 채용 공고가 3건입니다.
3. 최소 요구되는 연차는 0년(신입사원)입니다.
4. VR 직무에서의 경력별 채용 특징 및 트렌드를 분석해보면, 회사들은경험이 있는 인력을 선호하는 경향이 있지만, 동시에 신입사원의 채용에도 열려 있음을 확인할 수 있습니다. 이를 통해 VR 분야에서는 신기술의 급속한 발전으로 인해 새로운 아이디어와 열정을 가진 신입사원도 활용하려는 추세임을 알 수 있습니다. 그러나 가장 선호하는 인력은 역시 3년의 경력을 가진 사람들로, VR 분야의 전문 기능을 충실히 수행할 수 있다는 점에서 이러한 경향이 나타나는 것으로 보입니다. 그 외에도 2년과 5년차의 경력을 요구하는 채용 공고가 많은 것으로 보아 경력의 폭과 깊이를 중요하게 생각하는 기업들이 많은 것으로 해석할 수 있습니다.
✅ VR 직무 career 업데이트 완료: ['career_charts/VR.png', '1. 가장 많이 요청된 경력 연차는 3년으로, 해당 채용 공고는 총 5건입니다.\n2. 두 번째로 많이 요청된 경력 연차는 0년(신입사원), 2년, 5년으로, 각각 해당 채용 공고가 3건입니다.\n3. 최소 요구되는 연차는 0년(신입사원)입니다.\n4. VR 직무에서의 경력별 채용 특징 및 트렌드를 분석해보면, 회사들은경험이 있는 인력을 선호하는 경향이 있지만, 동시에 신입사원의 채용에도 열려 있음을 확인할 수 있습니다. 이를 통해 VR 분야에서는 신기술의 급속한 발전으로 인해 새로운 아이디어와 열정을 가진 신입사원도 활용하려는 추세임을 알 수 있습니다. 그러나 가장 선호하는 인력은 역시 3년의 경력을 가진 사람들로, VR 분야의 전문 기능을 충실히 수행할 수 있다는 점에서

In [ ]:
# vr 직무 완벽 완료 (그래프와 gpt4 설명 부합)
import openai
import sqlite3
import ast  # 문자열로 저장된 리스트 변환용

# ✅ OpenAI API 키 설정


In [ ]:
import openai
import sqlite3
import ast  # 문자열로 저장된 리스트 변환용

# ✅ OpenAI API 키 설정

# ✅ SQLite 데이터베이스 연결
db_path = "c:/Users/asia/Desktop/asia_camp_2024/final_project_asia/3.ETL_DB_Load(Career,Language)/asia (2).db"
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# ✅ SQL 쿼리 실행 (VR 직무 제외)
sql_query = """
WITH career_data AS (
    SELECT 'PM' AS category, career FROM processing WHERE duty LIKE '%PM%'
    UNION ALL
    SELECT '데이터 직무' AS category, career FROM processing WHERE duty LIKE '%데이터 직무%'
    UNION ALL
    SELECT '백엔드' AS category, career FROM processing WHERE duty LIKE '%백엔드%'
    UNION ALL
    SELECT '인프라 엔지니어' AS category, career FROM processing WHERE duty LIKE '%인프라 엔지니어%'
    UNION ALL
    SELECT '앱 개발자' AS category, career FROM processing WHERE duty LIKE '%앱 개발자%'
    UNION ALL
    SELECT '게임' AS category, career FROM processing WHERE duty LIKE '%게임%'
    UNION ALL
    SELECT 'AI' AS category, career FROM processing WHERE duty LIKE '%AI%'
    UNION ALL
    SELECT '임베디드' AS category, career FROM processing WHERE duty LIKE '%임베디드%'
    UNION ALL
    SELECT '프론트 엔드' AS category, career FROM processing WHERE duty LIKE '%프론트 엔드%'
    UNION ALL
    SELECT 'QA' AS category, career FROM processing WHERE duty LIKE '%QA%'
    UNION ALL
    SELECT '데이터 분석' AS category, career FROM processing WHERE duty LIKE '%데이터 분석%'
    UNION ALL
    SELECT '시스템' AS category, career FROM processing WHERE duty LIKE '%시스템%'
    UNION ALL
    SELECT '블록체인' AS category, career FROM processing WHERE duty LIKE '%블록체인%'
    UNION ALL
    SELECT 'ERP' AS category, career FROM processing WHERE duty LIKE '%ERP%'
    UNION ALL
    SELECT '언어별 개발자' AS category, career FROM processing WHERE duty LIKE '%언어별 개발자%'
)

SELECT
    category AS 직무,
    career AS 연차,
    COUNT(*) AS 채용_공고_개수
FROM career_data
GROUP BY category, career
ORDER BY category, career;
"""
cursor.execute(sql_query)
career_data = cursor.fetchall()

# ✅ 직무별 career 데이터 저장
job_career_dict = {}
for duty, career, job_count in career_data:
    if duty not in job_career_dict:
        job_career_dict[duty] = []
    job_career_dict[duty].append((career, job_count))  # ✅ 연차와 채용 건수 저장

# ✅ 모든 직무에 대해 GPT-4 분석 수행
for duty, career_info in job_career_dict.items():
    # ✅ 기존 career 컬럼 값 조회
    cursor.execute("SELECT career FROM duty_analysis WHERE duty = ?;", (duty,))
    existing_career_data = cursor.fetchone()

    # ✅ 연차별 데이터 변환
    if career_info:
        job_details = "\n".join([
            f"- {'신입사원' if c[0] == '0' else f'{c[0]}년차'}: {c[1]}건"
            for c in career_info
        ])
    else:
        job_details = f"{duty} 직무의 채용 데이터가 없습니다."

    print(f"🎯 {duty} 직무 데이터 확인:\n{job_details}")

    # ✅ GPT-4 프롬프트 생성
    prompt = f"""
    "{duty}" 직무의 채용 공고 데이터를 분석해주세요.

    [직무별 연차별 채용 공고 개수]
    {job_details}

    위 데이터를 기반으로 다음을 분석해주세요:
    1. 가장 많이 요청된 경력 연차와 해당 채용 공고 수
    2. 두 번째로 많이 요청된 경력 연차와 해당 채용 공고 수
    3. 최소 요구되는 연차
    4. {duty} 직무에서의 경력별 채용 특징 및 트렌드 분석

    ❗ 단, "신입사원"이라는 표현을 사용하며, "경력 0년차"라는 표현은 하지 마세요.
    ❗ 모든 문장은 "입니다"로 끝나도록 통일해주세요.
    ❗ 설명은 간결하게 요약해서 제공해주세요.
    ❗ "신입사원"이라는 단어가 변형되지 않도록 정확하게 출력해주세요.
    """

    try:
        # ✅ GPT-4 API 호출
        response = openai.chat.completions.create(
            model="gpt-4",
            messages=[{"role": "user", "content": prompt}]
        )
        gpt4_summary = response.choices[0].message.content.strip()

        print(f"✅ {duty} 직무 최종 응답:\n{gpt4_summary}")

        # ✅ 기존 career 컬럼 값 가져오기 (이미지 경로 유지)
        if existing_career_data and existing_career_data[0]:
            try:
                career_list = ast.literal_eval(existing_career_data[0])  # 문자열 → 리스트 변환
                if isinstance(career_list, list) and career_list:
                    image_path = career_list[0]  # 기존 이미지 경로 유지
                    career_list = [image_path, gpt4_summary]  # GPT 설명 업데이트
                else:
                    career_list = [gpt4_summary]  # 기존 데이터가 리스트가 아니면 새로 생성
            except:
                career_list = [gpt4_summary]  # 변환 실패 시 GPT 설명만 추가
        else:
            career_list = [gpt4_summary]  # 기존 값이 없으면 새 리스트 생성

        # ✅ 업데이트할 값 변환 (리스트 → 문자열)
        updated_text = str(career_list)

        # ✅ `duty_analysis` 테이블에서 해당 직무의 `career` 값 업데이트
        cursor.execute("UPDATE duty_analysis SET career = ? WHERE duty = ?;", (updated_text, duty))
        conn.commit()

        print(f"✅ {duty} 직무 career 업데이트 완료: {updated_text}")

    except Exception as e:
        print(f"❌ {duty} 직무 요약 실패: {e}")

# ✅ DB 연결 종료
conn.close()


🎯 AI 직무 데이터 확인:
- 0년차: 90건
- 1년차: 45건
- 2년차: 75건
- 3년차: 132건
- 4년차: 12건
- 5년차: 63건
- 6년차: 8건
- 7년차: 13건
- 8년차: 4건
- 10년차: 7건
✅ AI 직무 최종 응답:
1. 가장 많이 요청된 경력 연차는 3년차입니다. 해당 채용 공고 수는 132건입니다.
2. 두 번째로 많이 요청된 경력 연차는 0년차, 즉 신입사원입니다. 해당 채용 공고 수는 90건입니다.
3. 최소 요구되는 연차는 0년차, 즉 신입사원입니다.
4. AI 직무에서의 채용 특징 및 트렌드 분석 결과, 요구되는 경력 연차가 높을수록 채용 공고의 수는 줄어드는 경향을 보이고 있습니다. 3년차를 기점으로 그 이후로는 채용 공고 수가 현저하게 줄어드는데, 이는 AI 분야에서의 빠른 기술 변화를 따라가기 위해서는 유연성과 학습 능력이 중요하며, 경력보다는 관련 기술 및 분야에 대한 직접적인 경험과 지식을 중시하는 경향으로 해석될 수 있습니다. 신입사원에 대한 채용 공고 수가 높은 것도 이러한 트렌드를 반영한 것으로 보입니다.
✅ AI 직무 career 업데이트 완료: ['career_charts/AI.png', '1. 가장 많이 요청된 경력 연차는 3년차입니다. 해당 채용 공고 수는 132건입니다.\n2. 두 번째로 많이 요청된 경력 연차는 0년차, 즉 신입사원입니다. 해당 채용 공고 수는 90건입니다.\n3. 최소 요구되는 연차는 0년차, 즉 신입사원입니다.\n4. AI 직무에서의 채용 특징 및 트렌드 분석 결과, 요구되는 경력 연차가 높을수록 채용 공고의 수는 줄어드는 경향을 보이고 있습니다. 3년차를 기점으로 그 이후로는 채용 공고 수가 현저하게 줄어드는데, 이는 AI 분야에서의 빠른 기술 변화를 따라가기 위해서는 유연성과 학습 능력이 중요하며, 경력보다는 관련 기술 및 분야에 대한 직접적인 경험과 지식을 중시하는 경향으로 해석될 수 있습니다. 신입사원에 대한 채용 공고 수가 높은 것도 이러한 트렌드를 반영한 것으로